In [1]:
import sys
sys.path.append("..")
from VAE.vae import *

In [2]:
config_space = ConfigurationSpace(
                {'input_dropout': 0.1, 'intermediate_activation': "relu", 'intermediate_dimension': 10,
                'intermediate_layers': 2, 'latent_dimension': 1, 'learning_rate': 0.0001,
                'original_dim': 825000, 'solver': 'nadam'}
            )
config = config_space.get_default_configuration()

In [3]:
model = FIA_VAE(config)

In [4]:
model.summary()

Model: "fia_vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_intermediate            │ ?                      │     8,250,065 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_mu (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_sigma (Dense)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_reparametrization        │ ?                      │   0 (unbuilt) │
│ (Sampling)                      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ ?                      │     9,075,070 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,325,135 (66.09 MB)

 Trainable params: 17,325,135 (66.09 MB)

 Non-trainable params: 0 (0.00 B)

## Saving

In [5]:
keras.saving.save_model(model, "../../runs/VAE/training/test.keras")

/mnt/lustre/groups/link/linca945/.conda/envs/FIA_VAE/lib/python3.11/site-packages/keras/src/saving/saving_api.py:100: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  saving_lib.save_model(model, filepath)


In [6]:
model.save_weights("../../runs/VAE/training/test.weights.h5")

## Loading

In [7]:
model2 = keras.saving.load_model("../../runs/VAE/training/test.keras")

In [8]:
model2.load_weights("../../runs/VAE/training/test.weights.h5")

## Testing

In [9]:
data_dir  = os.path.normpath(os.path.join(os.getcwd(), "../../runs/FIA/Com8_grown_together/oms"))
data = read_data(data_dir, verbosity=0)

In [10]:
X = np.random.normal(0.0, 1.0, size=(5, 825000))

In [15]:
callbacks = [ keras.callbacks.ModelCheckpoint( filepath="../../runs/VAE/training/test_{epoch}.keras",
                                                       save_best_only=True, monitor="val_loss",
                                                       verbose=1 )]
callbacks.append( TensorBoard(log_dir="../runs/VAE/training", update_freq='epoch') )

In [16]:
model2.fit(X, X, validation_split=0.2, batch_size=2, epochs=10, callbacks=callbacks)

Epoch 1/10


ValueError: Expected scalar shape, saw shape: (2,).

In [ ]:
loss, kl_loss, recon_loss = model2.evaluate(data, data, batch_size=68)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - kl_loss: -0.0000e+00 - loss: 1.1580e-06 - reconstruction_loss: 1.1580e-06


In [ ]:
model2.predict(data)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)